In [ ]:
#install bark
!pip install git+https://github.com/suno-ai/bark.git

In [17]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tue Oct 31 11:19:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   50C    P5    22W / 170W |    793MiB / 12288MiB |     27%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 30%   

In [ ]:
!pip install --upgrade --quiet pip
!pip install --quiet git+https://github.com/huggingface/transformers.git

In [18]:
from transformers import BarkModel

model = BarkModel.from_pretrained("suno/bark")

/home/theeseus/anaconda3/envs/bark/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [19]:
model

BarkModel(
  (semantic): BarkSemanticModel(
    (input_embeds_layer): Embedding(129600, 1024)
    (position_embeds_layer): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x BarkBlock(
        (layernorm_1): BarkLayerNorm()
        (layernorm_2): BarkLayerNorm()
        (attn): BarkSelfAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (att_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): BarkMLP(
          (in_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
          (gelu): GELU(approximate='none')
        )
      )
    )
    (layernorm_final): BarkLayerNorm()
    (lm_head): Linear(in_feat

In [20]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [21]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("suno/bark")

In [22]:
text_prompt = "Let's try generating speech, with Bark, a text-to-speech model"
inputs = processor(text_prompt)

# generate speech
speech_output = model.generate(**inputs.to(device))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [23]:
from IPython.display import Audio

sampling_rate = model.generation_config.sample_rate
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [24]:
import scipy

scipy.io.wavfile.write("bark_out.wav", rate=sampling_rate, data=speech_output[0].cpu().numpy())

In [30]:
import numpy as np

# Select the voice preset to be used for text-to-speech synthesis
# You can choose between two options, and one is commented out
#voice_preset = "v2/en_speaker_1"
voice_preset = "v2/en_speaker_9"

# Define the text prompt to be converted into speech
text_prompt = """  
In our quest to find life beyond Earth, recent astronomical breakthroughs have rekindled our fascination with the cosmos... 
Kepler's pioneering mission revealed a galaxy teeming with exoplanets, some residing in their star's habitable zones... 
Notable findings include TOI 700 d, TRAPPIST-1's seven planets, and Kepler-442b and Kepler-186f...
The James Webb Space Telescope, launched in 2021, is poised to further our search by analyzing exoplanet atmospheres and biosignatures...
As we explore these Earth-like candidates, we're on the brink of discovering a universe rich with the potential for extraterrestrial life...
"""

# Define a function to generate audio for a given text chunk
def generate_audio_for_chunk(text_chunk, processor, model, voice_preset, device):
    # Prepare inputs for the text-to-speech synthesis
    inputs = processor(text_chunk, voice_preset=voice_preset)
    # Generate speech for the chunk using the TTS model
    speech_output = model.generate(**inputs.to(device))
    return speech_output[0].cpu().numpy()

# Define a function to break the input text into manageable chunks
def break_into_chunks(text, max_tokens=15):
    # Split the text into sentences and remove empty sentences
    sentences = [sentence.strip() for sentence in text.split('.') if sentence]
    chunks = []
    current_chunk = sentences[0]

    for sentence in sentences[1:]:
        # Create a combined version of the current chunk and the next sentence
        combined = current_chunk + ' ' + sentence

        # Check if appending the next sentence doesn't exceed the specified max_tokens limit
        if len(combined.split()) <= max_tokens:
            # If it doesn't exceed, update the current_chunk with the combined text
            current_chunk = combined
        else:
            # If it does exceed, append the current_chunk to the list of chunks
            chunks.append(current_chunk)
            # Start a new current_chunk with the current sentence
            current_chunk = sentence

    # Append the last current_chunk after processing all sentences
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

# Break the long text_prompt into manageable chunks
text_chunks = break_into_chunks(text_prompt)

# Generate audio for each chunk of text
all_speech_outputs = [generate_audio_for_chunk(chunk, processor, model, voice_preset, device) for chunk in text_chunks]

# Concatenate the audio outputs from all the text chunks into a single audio stream
concatenated_audio = np.concatenate(all_speech_outputs)

# Play the concatenated audio (assuming you have a mechanism in your environment to play audio)
Audio(concatenated_audio, rate=sampling_rate)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generati